In [75]:
import pandas as pd
import pickle
import math
from transformers import pipeline
from tqdm import tqdm

In [2]:
data = pd.read_csv('../data/wiki_movie_plots_deduped.csv')

In [3]:
data.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [6]:
print(data.shape)

(34886, 8)


In [28]:
filtered_data = data[data['Genre'] != 'unknown']
filtered_data = filtered_data[filtered_data['Cast'].notna()]
filtered_data = filtered_data[filtered_data['Director'] != 'Unknown']

In [29]:
filtered_data.head(10)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
13,1907,Daniel Boone,American,Wallace McCutcheon and Ediwin S. Porter,"William Craven, Florence Lawrence",biographical,https://en.wikipedia.org/wiki/Daniel_Boone_(19...,Boone's daughter befriends an Indian maiden as...
15,1907,Laughing Gas,American,Edwin Stanton Porter,"Bertha Regustus, Edward Boulden",comedy,https://en.wikipedia.org/wiki/Laughing_Gas_(fi...,The plot is that of a black woman going to the...
16,1908,The Adventures of Dollie,American,D. W. Griffith,"Arthur V. Johnson, Linda Arvidson",drama,https://en.wikipedia.org/wiki/The_Adventures_o...,On a beautiful summer day a father and mother ...
17,1908,The Black Viper,American,D. W. Griffith,D. W. Griffith,drama,https://en.wikipedia.org/wiki/The_Black_Viper,A thug accosts a girl as she leaves her workpl...
18,1908,A Calamitous Elopement,American,D.W. Griffith,"Harry Solter, Linda Arvidson",comedy,https://en.wikipedia.org/wiki/A_Calamitous_Elo...,A young couple decides to elope after being ca...
19,1908,The Call of the Wild,American,D. W. Griffith,Charles Inslee,adventure,https://en.wikipedia.org/wiki/The_Call_of_the_...,A white girl (Florence Lawrence) rejects a pro...
21,1908,The Fight for Freedom,American,D. W. Griffith,"Florence Auer, John G. Adolfi",western,https://en.wikipedia.org/wiki/The_Fight_for_Fr...,The film opens in a town on the Mexican border...
22,1909,At the Altar,American,D. W. Griffith,Marion Leonard,drama,https://en.wikipedia.org/wiki/At_the_Altar,A rejected admirer sets up a trap to kill his ...
23,1909,A Drunkard's Reformation,American,D. W. Griffith,Arthur V. Johnson,drama,https://en.wikipedia.org/wiki/A_Drunkard%27s_R...,"John Wharton, the husband of a true and trusti..."
26,1910,An Arcadian Maid,American,D.W. Griffith,"Mary Pickford, Mack Sennett",drama,https://en.wikipedia.org/wiki/An_Arcadian_Maid,Mary Pickford plays Priscilla an unemployed ma...


In [30]:
print(filtered_data.shape)

(27832, 8)


In [104]:
director_dict = filtered_data['Director'].value_counts().to_dict()
origin_dict = filtered_data['Origin/Ethnicity'].value_counts().to_dict()

In [65]:
cast_dict = {}
genre_dict= {}
origin_dict = {}
year_dict = {}
index_list = []
new_df = pd.DataFrame(columns=['Release Year', 'title', 'origin', 'director', 'cast', 'genre', 'plot'])

In [66]:
for i in range(filtered_data.shape[0]):
    if director_dict[filtered_data.iloc[i]['Director']] < 2:
        continue
    # Process casts
    row_data = filtered_data.iloc[i]
    cast_list = [x.strip() for x in row_data['Cast'].split(',')]
    for cast in cast_list:
        if cast in cast_dict:
            cast_dict[cast] += 1
        else:
            cast_dict[cast] = 1
    tmp_list = [x.strip() for x in row_data['Genre'].split(',')]
    genre_list = []
    for genre in tmp_list:
        genre_list.extend(genre.split('/'))
    
    for genre in genre_list:
        if genre in genre_dict:
            genre_dict[genre] += 1
        else:
            genre_dict[genre] = 1
    new_df.loc[len(new_df.index)] = [row_data['Release Year'], row_data['Title'], row_data['Origin/Ethnicity'], row_data['Director'], cast_list, genre_list, row_data['Plot']]

In [62]:
for k, v in cast_dict.items():
    if v != 1:
        print(k, v)

Arthur V. Johnson 2
Linda Arvidson 2
Mary Pickford 16
Mack Sennett 3
Henry B. Walthall 10
William Russell 2
Lillian Gish 20
Mabel Normand 12
Ford Sterling 4
Mae Marsh 9
Blanche Sweet 5
Donald Crisp 16
Lilian Gish 2
Edward Abeles 2
Charlie Chaplin 28
Mack Swain 3
Phyllis Allen 5
Theodore Roberts 3
Edgar Kennedy 4
Alice Davenport 3
Chester Conklin 3
Fritz Schade 2
Dorothy Gish 8
Al St. John 6
Fatty Arbuckle 8
Annette Kellerman 2
Jane Darwell 7
House Peters 3
Marie Dressler 12
Sessue Hayakawa 7
Geraldine Farrar 4
Wallace Reid 9
Charles Chaplin 9
Victor Moore 7
Harold Lloyd 35
Louise Glaum 7
James Neill 2
Edna Purviance 8
Robert Harron 4
Bebe Daniels 22
Douglas Fairbanks 40
Jewel Carmen 2
Theda Bara 5
Harry Carey 16
Molly Malone 2
Gertrude Astor 3
Sr. 4
Thurston Hall 7
Jack Pickford 5
'Fatty' Arbuckle 4
Buster Keaton 34
Mabel Taliaferro 2
Mary Miles Minter 7
Allan Forrest 5
Marjorie Daw 3
Douglas MacLean 3
Ann Little 3
Noah Beery 13
Mae Murray 6
Lew Cody 5
Edith Roberts 3
Hallam Cooley 3
E

In [63]:
for k, v in genre_dict.items():
    if v!=1:
        print(k, v)

drama 5760
comedy 4376
adventure 617
western 870
horror 921
biographical drama 4
fantasy 288
adventure serial 3
historical drama 40
epic 15
historical 86
comedy short 46
biography 253
short comedy 3
mystery 383
crime drama 419
romance 1135
war drama 55
comedy drama 75
spy 67
romantic comedy 383
biopic 53
short 21
drama romance 3
melodrama 30
swashbuckler 11
crime 761
romance drama 21
crime comedy 36
comedy western 5
war 349
thriller 945
dramatic comedy 4
romantic drama 91
crime thriller 48
biblical drama 5
musical 638
musical comedy 146
comedy mystery 5
detective 6
romance musical 4
documentary 18
operetta 3
costume drama 14
prison drama 5
spy drama 6
action 1292
science fiction 386
film noir 333
sports 57
romantic 10
sci-fi 258
historical comedy 3
comedy romance 6
sport 4
exploitation 6
comedy-drama 109
serial 58
sports drama 16
music 9
action adventure 13
suspense 109
gangster 5
biographical 20
biblical 4
bio-pic 18
adventure drama 3
musical romance 2
musical fantasy 3
comedy thrille

In [73]:
new_df.iloc[0]['plot']

"On a beautiful summer day a father and mother take their daughter Dollie on an outing to the river. The mother refuses to buy a gypsy's wares. The gypsy tries to rob the mother, but the father drives him off. The gypsy returns to the camp and devises a plan. They return and kidnap Dollie while her parents are distracted. A rescue crew is organized, but the gypsy takes Dollie to his camp. They gag Dollie and hide her in a barrel before the rescue party gets to the camp. Once they leave the gypsies and escapes in their wagon. As the wagon crosses the river, the barrel falls into the water. Still sealed in the barrel, Dollie is swept downstream in dangerous currents. A boy who is fishing in the river finds the barrel, and Dollie is reunited safely with her parents."

In [106]:
pipe = pipeline("summarization", model="Falconsai/text_summarization", batch_size=30)

In [113]:
summary_list = []

In [ ]:
for i in tqdm(range(0, new_df.shape[0], 30)):
    plots = list(new_df['plot'].iloc[i:i+30])
    summaries = [x['summary_text'] for x in pipe(plots, max_length=130, min_length=10, do_sample=False)]
    print(summaries)
    summary_list.extend(summaries)


  0%|                                                                                         | 1/714 [01:56<23:01:35, 116.26s/it]

["Dollie is swept downstream in dangerous currents . The mother refuses to buy a gypsy's wares, but the father drives him off . They return and kidnap the mother while her parents are distracted .", 'thug vows revenge and attacks the girl and her rescuer again . This time they succeed in kidnapping the rescuer . They track the ruffians down to a cabin in the mountains where the gang has trapped their victim .', "a young couple decides to elope after being caught in the midst of a romantic moment by the woman's angry father . A thief discovers their plans and hides in their trunk and waits for the right moment to steal their belongings .", 'Florence Lawrence is the most popular among the Biograph Company\'s anonymous stock company players . By 1909, Florence Lawrence was known the world over as "The Biograph Girl"', 'a poker game is going on in a town on the Mexican border . One of the players cheats and is shot dead by another player, Pedro . While Pedro hides, his wife Juanita is arre

In [112]:
print(len(summary_list))

30
